# 5. FFNN

In [2]:
from utils import load_data
import numpy as np
import pandas as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

VEC_SIZE = 300

dataset = load_data('cleaned')

In [8]:
from vectors import vectorize

data = np.vstack(dataset.data.apply(vectorize))
target = dataset.target

In [55]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [56]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.nn.functional import relu

torch.manual_seed(0)

In [57]:
train_data = torch.from_numpy(X_train)
train_target = torch.from_numpy(y_train_enc)
test_data = torch.from_numpy(X_test)
test_target = torch.from_numpy(y_test_enc)

In [61]:
class RequestClassifier(nn.Module):
    def __init__(self, input_size, num_class):
        super(RequestClassifier, self).__init__()
        self.linear1 = nn.Linear(input_size, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, num_class)
        
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        
        self.bn1 = nn.BatchNorm1d(128, track_running_stats=False)
        self.bn2 = nn.BatchNorm1d(64, track_running_stats=False)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.bn1(x)
        x = relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = relu(x)
        x = self.linear3(x)
        
        return x


def train(model, loss, train_data, train_target, epochs = 1):
    # optimizer = optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
    lr = 0.01 # 3e-4
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for n in range(1, epochs + 1, 1):
        optimizer.zero_grad()
        
        train_output = model(train_data)
        
        train_error = loss(train_output, train_target)
        train_error.backward()
        
        if n % 50 == 0 or n == 1 or n == epochs:
            print("Step = {}/{} Error = {}".format(n, epochs, train_error.item()))
        
        optimizer.step()
    
    return model

In [62]:
# balance weights
#
# from collections import Counter
# c = Counter(y_train_enc)
# biggest = max(c.values())
# weights = torch.tensor([biggest / n for i,n in sorted(c.items())])
# loss = nn.CrossEntropyLoss(weight=weights)

loss = nn.CrossEntropyLoss()

In [63]:
model = RequestClassifier(VEC_SIZE, num_class = len(le.classes_))

train(model, loss, train_data, train_target, 500)

Step = 1/500 Error = 5.419687747955322
Step = 50/500 Error = 1.5266588926315308
Step = 100/500 Error = 0.9168763160705566
Step = 150/500 Error = 0.6206905841827393
Step = 200/500 Error = 0.444354385137558
Step = 250/500 Error = 0.3141119182109833
Step = 300/500 Error = 0.3009711802005768
Step = 350/500 Error = 0.29438549280166626
Step = 400/500 Error = 0.11612709611654282
Step = 450/500 Error = 0.11335896700620651
Step = 500/500 Error = 0.06596121937036514


RequestClassifier(
  (linear1): Linear(in_features=300, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=204, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
)

In [64]:
y_pred = model(train_data)
y_pred = torch.max(y_pred, 1).indices
y_pred

pd.DataFrame(classification_report(train_target, y_pred, output_dict=True))

,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,accuracy,macro avg,weighted avg
precision,1.000000,1.0,0.995717,0.995098,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,0.987654,0.990099,0.992481,1.0,0.948276,0.993282,0.994353,0.993318
recall,0.994792,1.0,0.989362,1.000000,1.0,1.0,1.0,1.0,1.0,0.997738,...,1.0,1.0,0.993789,0.995025,1.000000,1.0,0.982143,0.993282,0.994405,0.993282
f1-score,0.997389,1.0,0.992529,0.997543,1.0,1.0,1.0,1.0,1.0,0.998867,...,1.0,1.0,0.990712,0.992556,0.996226,1.0,0.964912,0.993282,0.994330,0.993282
support,192.000000,36.0,470.000000,203.000000,49.0,62.0,61.0,33.0,96.0,442.000000,...,54.0,46.0,161.000000,402.000000,132.000000,43.0,56.000000,0.993282,42277.000000,42277.000000


In [65]:
y_pred = model(test_data)
y_pred = torch.max(y_pred, 1).indices
y_pred

pd.DataFrame(classification_report(test_target, y_pred, output_dict=True))

,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,accuracy,macro avg,weighted avg
precision,0.438596,0.0,0.459330,0.145631,0.592593,0.227273,0.440000,0.285714,0.527778,0.629808,...,0.178571,0.461538,0.265306,0.391667,0.428571,0.571429,0.222222,0.516711,0.387443,0.521746
recall,0.420168,0.0,0.424779,0.189873,0.666667,0.227273,0.343750,0.111111,0.593750,0.642157,...,0.172414,0.272727,0.309524,0.427273,0.338710,0.380952,0.272727,0.516711,0.382981,0.516711
f1-score,0.429185,0.0,0.441379,0.164835,0.627451,0.227273,0.385965,0.160000,0.558824,0.635922,...,0.175439,0.342857,0.285714,0.408696,0.378378,0.457143,0.244898,0.516711,0.381174,0.517537
support,119.000000,14.0,226.000000,79.000000,24.000000,22.000000,32.000000,18.000000,32.000000,204.000000,...,29.000000,22.000000,84.000000,220.000000,62.000000,21.000000,22.000000,0.516711,20824.000000,20824.000000


**Далі:** [Візуалізація](06-vectors-tsne.ipynb)